# RAG application for "A Guide for First-Time Parents"

## Install tika and parse PDF file

- Install libraries
- Download PDF from the website [The Asian Parent](https://th.theasianparent.com/%E0%B8%84%E0%B8%B9%E0%B9%88%E0%B8%A1%E0%B8%B7%E0%B8%AD%E0%B8%94%E0%B8%B9%E0%B9%81%E0%B8%A5%E0%B8%A5%E0%B8%B9%E0%B8%81)
- Create index using llama index
- Clean text (using a simple created function)

In [13]:
%%capture
!pip install faker google-cloud-aiplatform
!pip install llama-index-llms-vertex

In [64]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from dotenv import load_dotenv
load_dotenv()

# global default
Settings.embed_model = OpenAIEmbedding()

documents = SimpleDirectoryReader(input_files=["baby_0_3.pdf"]).load_data()

len(documents)

86

In [93]:
index = VectorStoreIndex.from_documents(documents)

In [92]:
engine = index.as_query_engine()

In [83]:
retriever = index.as_retriever()

## Prompting using RAG

- Embed text chunks with and store using `llama index`
- Find the closest text chunks
- Add information and perform RAG

In [84]:
from google.oauth2 import service_account
from google.cloud import aiplatform
from vertexai.language_models import TextGenerationModel, TextEmbeddingModel

service_account_path = "service_account.json"

credentials = service_account.Credentials.from_service_account_file(service_account_path)
aiplatform.init(project=credentials.project_id, credentials=credentials)

In [85]:
question = "ถ้าลูกดูดนิ้วต้องทำอย่างไรบ้าง"

nodes = retriever.retrieve(question)

retrieved_chunk = [node.text for node in nodes]

In [86]:
retrieved_chunk[0]

'ลูกสามารถสื่อสารได้ว่าต้องการจะปัสสาวะหรืออุจจาระ\t อาจเป็นการสื่อสารด้วยค�าพูดหรือ\nใช้ท่าทาง\t 6.\t ลูกให้ความร่วมมือและอยากฝึกบนพื้นฐานของความต้องการเป็นตัวของตัวเองซึ่งสอดคล้องกับพัฒนาการของลูกวัยนี้\t 7.\t ฝึกภายใต้บรรยากาศและความสัมพันธ์อันดีระหว่างพ่อแม่และลูก\n\tขั้นตอนในการฝึกลูกขับถ่าย\n\t 1.\t ใช้ค�าแทนการปัสสาวะหรืออุจจาระง่ายๆ\tกับลูก\tเช่น\tฉี่\tอึ\tกระโถน\tส้วม\t\t 2.\t เปิดโอกาสให้ลูกได้มีส่วนร่วมในการเลือกวิธีการฝึก\t เช่น\tเลือกว่าจะนั่งกระโถนหรือนั่งชักโครก\n\t 3.\t ในช่วงต้น\t ควรสร้างความคุ้นเคยโดยให้ลูกนั่งกระโถนหรือส้วมโดยยังไม่ต้องถอดเสื้อผ้า\t และ\nพ่อแม่อาจลองนั่งให้ลูกดูเป็นตัวอย่าง \tเมื่อคุ้นเคยแล้วเริ่มให้นั่งโดยถอดกางเกงและให้นั่งหลังมื้ออาหาร\nอย่างสม�่าเสมอ\t 4.\t ให้ก�าลังใจและแสดงความชื่นชมเมื่อลูกท�าได้\t อย่าเล็งผลเลิศในทันทีที่ฝึก\t ไม่คาดคั้นลูกเมื่อ\nลูกท�าไม่ได้\t การฝึกขับถ่ายนั้นจะไม่มีการลงโทษหรือว่ากล่าวตักเตือนที่รุนแรง\t เพราะนอกจากจะไม่เกิด\nประโยชน์อันใดแล้วยังก่อให้เกิดโทษ \tลูกจะต่อต้านและอาจเกิดปัญหากลั้นปัสสาวะหรือท้องผูกตามมา \t\nห

Compare the prompt with (RAG) and using llama index chat engine

In [87]:
def complete_text(prompt: str):
    gen_model = TextGenerationModel.from_pretrained("text-bison")
    prompt = "You are a helpful assistant designed to generate output prompt for parent who are asking questions about newborn in Thai.\n" + prompt
    
    return gen_model.predict(prompt).text

In [88]:
prompt_with_context = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the given query in Thai. The answer should be concise and clear.
Query: {question}
Answer:
"""

prompt_no_context = f"""
Answer the given query in Thai. The answer should be concise and clear.
Query: {question}
Answer:
"""

In [89]:
output_no_rag = engine.query(question)
print(output_no_rag)

ให้ลูกนิ้วต้องการดูดนิ้ว ควรสร้างการตอบสนองด้วยการสัมผัสอ่อนโยน และส่งเสริมการเรียนรู้ด้วยการให้ลูกจับนิ้วของคุณ โดยไม่ควรใช้นิ้วหรือวัตถุอื่นทดแทนในขณะที่ลูกต้องการดูดนิ้ว


In [90]:
output_rag = complete_text(prompt_with_context)
print(output_rag)